In [1]:
from royalur import Board
from royalur.model import StandardBoardShape, Piece, PlayerState
from royalur.model.player import PlayerType
from royalur.rules.state import WaitingForRollGameState
#import cProfile
from royalur.lut.board_encoder import SimpleGameStateEncoding
import duckdb

In [11]:

import pandas as pd


char_to_player = {
    "L": PlayerType.LIGHT,
    "D": PlayerType.DARK,
}
"""
.DD 0-2
..D 3-5
.LD 6-8
..D 9-11
 L  12-14
 .  15-17
LD. 18-20
.D. 21-23
"""
light_path = [
    (1, 4),
    (1, 3),
    (1, 2),
    (1, 1),
    (2, 1),
    (2, 2),
    (2, 3),
    (2, 4),
    (2, 5),
    (2, 6),
    (2, 7),
    (2, 8),
    (1, 8),
    (1, 7),
]
common_path = [1, 4, 7, 10, 13, 16, 19, 22]
light_path = [9, 6, 3, 0, *common_path,  21, 18]
dark_path = [
    (3, 4),
    (3, 3),
    (3, 2),
    (3, 1),
    (2, 1),
    (2, 2),
    (2, 3),
    (2, 4),
    (2, 5),
    (2, 6),
    (2, 7),
    (2, 8),
    (3, 8),
    (3, 7),
]
dark_path = [11, 8, 5, 2,  *common_path,  23, 20]
rosette_index = [0, 2, 10, 18, 20]

light_path_map = {e: i for i, e in enumerate(light_path)}
dark_path_map = {e: i for i, e in enumerate(dark_path)}
CENTER_ROSETTE_POS = 10
LIGHT_TOP_ROSETTE_POS = 0
LIGHT_BOTTOM_ROSETTE_POS = 18
path_len = 14

light_rosette_index = [LIGHT_TOP_ROSETTE_POS, CENTER_ROSETTE_POS, LIGHT_BOTTOM_ROSETTE_POS]

char_to_path = {
    "L": light_path,
    "D": dark_path,
}
example_game = """
LD.
.D.
...
...
...
...
L..
..D
"""

def formatted(boardstr):
    boardstrings = boardstr
    if type(boardstr) == str:
        boardstrings = [boardstr]
    parts = []
    for i in range(0, len(boardstrings[0]), 3):
        if i == 12 or i == 15:
            parts.append(" ".join([" " + boardstr[i + 1] + " " for boardstr in boardstrings]))
            continue
        parts.append(" ".join([boardstr[i: i+3] for boardstr in boardstrings]))
    return '\n'.join(parts)

# remove newlines from example_game
example_game = example_game.replace("\n", "")
example_game

def overwrite(s, char, index):
    return s[:index] + char + s[index+1:]

def compute_available_moves(boardstr, light_score, light_pawns, dark_score, dark_pawns):
    possible_moves = [
        (
            boardstr,
            light_score,
            light_pawns,
            dark_score,
            dark_pawns,
            0,
            2,
            False,
        )
    ]
    can_add_pawn = light_pawns > 0
    # contains the index of the boardstr where the pawn is along with the poslut_transition
                light_pawns - 1,
                dark_score,
                dark_pawns,
                dice,
                1 if play_boardstr_index == LIGHT_TOP_ROSETTE_POS else 2,
                False
            ))
        for active_light_pawn_boardstr_index, active_light_pawn_path_index in active_light_pawns:
            if active_light_pawn_path_index + dice == path_len:
                # one more point
                new_boardstr = overwrite(boardstr, '.', active_light_pawn_boardstr_index)
                new_light_score = light_score + 1
                possible_moves.append((
                    new_boardstr,
                    new_light_score,
                    light_pawns,
                    dark_score,
                    dark_pawns,
                    dice,
                    2,
                    new_light_score == 7
                ))
            elif active_light_pawn_path_index + dice < path_len:
                play_boardstr_index = light_path[active_light_pawn_path_index + dice]
                # we play over a D pawn that is not on the center rosette
                if boardstr[play_boardstr_index] == "D" and play_boardstr_index != CENTER_ROSETTE_POS:
                    new_boardstr = overwrite(boardstr, '.', active_light_pawn_boardstr_index)
                    new_boardstr = overwrite(new_boardstr, 'L', play_boardstr_index)
                    possible_moves.append((
                        new_boardstr,
                        light_score,
                        light_pawns,(current_state INTEGER, next_state INTEGER, dice INTEGER, next_player INTEGER);
                        dice,
                        2,
                        False
                    ))
                elif boardstr[play_boardstr_index] == ".":
                    # we play over a free spot
                    new_boardstr = overwrite(boardstr, '.', active_light_pawn_boardstr_index)
                    new_boardstr = overwrite(new_boardstr, 'L', play_boardstr_index)
                    possible_moves.append((
                        new_boardstr,
                        light_score,
                        light_pawns,
                        dark_score,
                        dark_pawns,
                        dice,
                        1 if play_boardstr_index in light_rosette_index else 2,
                        False
                    ))

    return possible_moves

o = compute_available_moves(example_game, 0, 5, 0, 4)
pd.DataFrame(o, columns=["boardstr", "light_score", "light_pawns", "dark_score", "dark_pawns", "dice", "active_player", "light_won"])

,boardstr,light_score,light_pawns,dark_score,dark_pawns,dice,active_player,light_won
0,LD..D.............L....D,0,5,0,4,0,2,False
1,LD..D....L........L....D,0,4,0,4,1,2,False
2,.L..D.............L....D,0,5,0,5,1,2,False
3,LD..D..................D,1,5,0,4,1,2,False
4,LD..D.L...........L....D,0,4,0,4,2,2,False
5,.D..L.............L....D,0,5,0,5,2,2,False
6,LD.LD.............L....D,0,4,0,4,3,2,False
7,.D..D..L..........L....D,0,5,0,4,3,2,False
8,.D..D.....L.......L....D,0,5,0,4,4,1,False


In [38]:
print(formatted(example_game))
print()
print(formatted(o[2][0]))

LD.
.D.
...
...
 . 
 . 
L..
..D

.L.
.D.
...
...
 . 
 . 
L..
..D


In [43]:
from tqdm import tqdm
from joblib import Parallel, delayed
from joblib.externals.loky import set_loky_pickler


def embarrassing2(contents, callback, n_jobs=8):
    with tqdm(total=len(contents)) as progress_bar:

        def uuu(q):
            ret = callback(q)
            progress_bar.update()
            return ret

        set_loky_pickler('pickle')
        r = Parallel(n_jobs=n_jobs, prefer="processes")(
            delayed(uuu)(i) for i in contents
        )
        set_loky_pickler()
        return r

In [1]:
import duckdb
def do_query(query):
    with duckdb.connect('local2.db') as con:
        return con.execute(query).fetchdf()
def do_query_obj(query):
    with duckdb.connect('local2.db') as con:
        return con.execute(query).fetchall()

In [30]:
do_query("DROP TABLE IF EXISTS lut_transition;")
do_query("CREATE TABLE lut_transition(current_state INTEGER, next_state INTEGER, dice INTEGER, next_player INTEGER);")

,Count


In [2]:
do_query("""
         SET memory_limit='20GB';
         SELECT * FROM lut_transition LIMIT 100000 OFFSET 1000000000;
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [2]:
result = 200
divisions = 0

while result > 1:
    result /= 2
    print(result)
    divisions += 1

print(divisions)


100.0
50.0
25.0
12.5
6.25
3.125
1.5625
0.78125
8


In [55]:
do_query("CREATE INDEX current_state_index ON lut_transition(current_state);")
do_query("CREATE INDEX next_state_index ON lut_transition(next_state);")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [24]:
do_query("SELECT AVG(lut_value), stddev_pop(lut_value) FROM boardstate_lut_expanded_int_values WHERE B4 = 1;")

,avg(lut_value),stddev_pop(lut_value)
0,0.566065,0.143033


In [27]:
do_query("""
ATTACH 'dbname=postgres user=postgres password=mysecretpassword host=127.0.0.1' AS postgres_db (TYPE postgres);
COPY postgres_db.lut_transition FROM 'lut_transition.parquet';
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Error: Failed to copy data: ERROR:  could not extend file "base/5/16426.58" with FileFallocate(): No space left on device
HINT:  Check free disk space.
CONTEXT:  COPY lut_transition, line 1409739000


In [26]:
do_query("""
ATTACH 'dbname=postgres user=postgres password=mysecretpassword host=127.0.0.1' AS postgres_db (TYPE postgres);
COPY FROM DATABASE local2 TO postgres_db
""")

InternalException: INTERNAL Error: Error during PQputCopyData: server closed the connection unexpectedly
	This probably means the server terminated abnormally
	before or while processing the request.


In [ ]:
do_query("""
ATTACH 'dbname=postgres user=postgres password=mysecretpassword host=127.0.0.1' AS postgres_db (TYPE postgres);
COPY postgres_db.boardstate_lut_expanded_int_values FROM 'boardstates.parquet';
""")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Error: KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /home/raph/.cache/pypoetry/virtualenvs/ur-lut-visualizer-OffVFizb-py3.10/lib/python3.10/site-packages/traitlets/traitlets.py(708): __set__
  /tmp/ipykernel_37588/2866858935.py(3): do_query
  /tmp/ipykernel_37588/545005560.py(1): <module>
  /home/raph/.cache/pypoetry/virtualenvs/ur-lut-visualizer-OffVFizb-py3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3577): run_code
  /home/raph/.cache/pypoetry/virtualenvs/ur-lut-visualizer-OffVFizb-py3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3517): run_ast_nodes
  /home/raph/.cache/pypoetry/virtualenvs/ur-lut-visualizer-OffVFizb-py3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3334): run_cell_async
  /home/raph/.cache/pypoetry/virtualenvs/ur-lut-visualizer-OffVFizb-py3.10/lib/python3.10/site-packages/IPython/core/async_helpers.py(129): _pseudo_sync_runner
  /home/raph/.cache/pypoetry/virtualenvs/ur-lut-visualizer-OffVFizb-py3.10/lib/pyth

In [17]:
do_query("""
ATTACH 'dbname=postgres user=postgres password=mysecretpassword host=localhost' AS postgres_db (TYPE postgres);
SELECT * FROM postgres_query('postgres_db', 'DROP TABLE public.boardstate_lut_expanded_int_values;');
""")

ParserException: Parser Error: syntax error at or near "postgres_query"

In [32]:
do_query("""
SELECT * FROM lut_transition
INNER JOIN (SELECT lut_index FROM boardstate_lut_expanded_int_values WHERE B4 = 1) as lut_index
ON lut_transition.current_state = lut_index
         """)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Error: KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /home/raph/.cache/pypoetry/virtualenvs/ur-lut-visualizer-OffVFizb-py3.10/lib/python3.10/site-packages/traitlets/traitlets.py(708): __set__
  /tmp/ipykernel_22939/2866858935.py(3): do_query
  /tmp/ipykernel_22939/3892483226.py(1): <module>
  /home/raph/.cache/pypoetry/virtualenvs/ur-lut-visualizer-OffVFizb-py3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3577): run_code
  /home/raph/.cache/pypoetry/virtualenvs/ur-lut-visualizer-OffVFizb-py3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3517): run_ast_nodes
  /home/raph/.cache/pypoetry/virtualenvs/ur-lut-visualizer-OffVFizb-py3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py(3334): run_cell_async
  /home/raph/.cache/pypoetry/virtualenvs/ur-lut-visualizer-OffVFizb-py3.10/lib/python3.10/site-packages/IPython/core/async_helpers.py(129): _pseudo_sync_runner
  /home/raph/.cache/pypoetry/virtualenvs/ur-lut-visualizer-OffVFizb-py3.10/lib/pyt

In [25]:
do_query("SELECT AVG(lut_value), stddev_pop(lut_value) FROM boardstate_lut_expanded_int_values WHERE B4 = 2;")

,avg(lut_value),stddev_pop(lut_value)
0,0.474838,0.146902


In [30]:
do_query("SELECT AVG(lut_value), stddev_pop(lut_value) FROM boardstate_lut_expanded_int_values WHERE B4 = 0;")

,count_star()
0,59892498


In [44]:
import pandas as pd
from royalur.model import Roll


def quickroll(game, value):
    """
    Rolls the dice, and updates the state of the
    game accordingly. If a value is supplied, then
    the roll will have that value.
    """

    # Update the state of the game after rolling.
    game._states += game._rules.apply_roll(game._states[-1], Roll(value))

limit = 1000000
offset = 0
encoding = SimpleGameStateEncoding()
#profile = cProfile.Profile()
#profile.enable()
def compute(list_of_tup):
    b = Board(StandardBoardShape())
    insert_buffer = []
    for board_state, light_score, light_piece_left_to_play, dark_score, dark_piece_left_to_play, lut_index in list_of_tup:
        #light_pieces, dark_pieces = boardstate_string_to_board(index_cache, b, board_state, light_score, dark_score)

        #light_player_state = PlayerState(PlayerType.LIGHT, light_pieces, light_score)
        #dark_player_state = PlayerState(PlayerType.DARK, dark_pieces, dark_score)

        #gs = WaitingForRollGameState(
        #    b,
        #    light_player_state,
        #    dark_player_state,
        #    PlayerType.LIGHT,
        #)
        insert_buffer.append((lut_index, lut_index, 0, 2))
        moves = compute_available_moves(
            board_state,
            light_score,
            light_piece_left_to_play,
            dark_score,
            dark_piece_left_to_play,
        )
        for boardstr, light_score, light_pawns, dark_score, dark_pawns, dice, active_player, light_won in moves:
            # ["boardstr", "light_score", "light_pawns", "dark_score", "dark_pawns", "dice", "active_player", "light_won"]
            if dice == 0:
                continue

            for i in range(len(boardstr)):
                state = boardstr[i]
                p = None
                if state != ".":
                    p = Piece(char_to_player[state], path_map[state][i])
                b._pieces[i] = p
            light_player_state = PlayerState(PlayerType.LIGHT, light_pawns, light_score)
            dark_player_state = PlayerState(PlayerType.DARK, dark_pawns, dark_score)
            gs = WaitingForRollGameState(
                b,
                light_player_state,
                dark_player_state,
                PlayerType.LIGHT,
            )
            lut_state = encoding.encode_game_state(gs)
            if light_won:
                insert_buffer.append((lut_index, lut_state, dice, -1))
            else:
                insert_buffer.append((lut_index, lut_state, dice, active_player))
    return insert_buffer

path_map = {
    "L": light_path_map,
    "D": dark_path_map,
}

n = limit // 10  #chunk row size
def do_the_thing(compute, n, to_convert):
    list_of_list = [to_convert[i:i+n] for i in range(0,len(to_convert),n)]
    insert_buffers = embarrassing2(list_of_list, compute, n_jobs=10)
    # flatten insert_buffers
    insert_buffers = [x for sublist in insert_buffers for x in sublist]
    # create dataframe
    return pd.DataFrame(insert_buffers, columns=["current_state", "next_state", "dice", "next_player"])

while True:
    to_convert = do_query_obj(f"""
    SELECT boardstate, light_score, light_piece_left_to_play, dark_score, dark_piece_left_to_play, lut_index
    FROM boardstate_lut_expanded_int_values 
    WHERE light_score < 7 AND dark_score < 7 
    LIMIT {limit} OFFSET {offset}
    """)
    if len(to_convert) == 0:
        break
    offset += limit
    print(offset)
    do_the_thing(compute, n, to_convert)
    print("Inserting...")
    with duckdb.connect('local2.db') as con:
        con.execute("INSERT INTO lut_transition SELECT * FROM df")
    print("Done")
    break

#profile.disable()
#profile.print_stats()

1000000


  0%|          | 0/10 [00:00<?, ?it/s]


PicklingError: Could not pickle the task to send it to the workers.

In [6]:
do_query("SELECT dice, count(*) FROM lut_transition WHERE next_player = -1 GROUP BY dice LIMIT 10")

,dice,count_star()
0,1,21919
1,2,21919
2,3,13865
3,4,13865


In [6]:
df2 = do_query("SELECT * FROM lut_transition WHERE next_player = -1 AND dice = 4")
df2

,current_state,next_state,dice,next_player
0,33570368,33569984,4,-1
1,33570400,33570016,4,-1
2,33570384,33570000,4,-1
3,33570416,33570032,4,-1
4,33570376,33569992,4,-1
...,...,...,...,...
13860,167824384,167824000,4,-1
13861,167834752,167834368,4,-1
13862,167772544,167772160,4,-1
13863,167772576,167772192,4,-1


In [9]:
def print_states(boardstates1, boardstates2):
    for boardstate1, boardstate2, in zip(boardstates1, boardstates2):
        #print(boardstate1, boardstate2)
        print(formatted([boardstate1, boardstate2]))
        print()

In [14]:
def get_one_state(con, state):
    return con.execute(f"SELECT boardstate, lut_value FROM boardstate_lut_expanded_int_values WHERE lut_index = {state} LIMIT 1").fetchone()

with duckdb.connect('local2.db') as con:
    current_states = []
    lut_values_current = []
    for state in df2.current_state:
        s = get_one_state(con, state)
        current_states.append(s[0])
        lut_values_current.append(s[1])
    next_states = []
    lut_values_next = []
    for state in df2.next_state:
        s = get_one_state(con, state)
        next_states.append(s[0])
        lut_values_next.append(s[1])
print_states(current_states, next_states)

... ...
... ...
.D. .D.
... ...
 .   . 
 .   . 
.L. ...
... ...

... ...
... ...
.D. .D.
... ...
 .   . 
 .   . 
.L. ...
..D ..D

... ...
... ...
.D. .D.
... ...
 .   . 
 .   . 
.LD ..D
... ...

... ...
... ...
.D. .D.
... ...
 .   . 
 .   . 
.LD ..D
..D ..D

... ...
... ...
.D. .D.
..D ..D
 .   . 
 .   . 
.L. ...
... ...

... ...
... ...
.D. .D.
..D ..D
 .   . 
 .   . 
.L. ...
..D ..D

... ...
... ...
.D. .D.
..D ..D
 .   . 
 .   . 
.LD ..D
... ...

... ...
... ...
.D. .D.
..D ..D
 .   . 
 .   . 
.LD ..D
..D ..D

... ...
... ...
.DD .DD
... ...
 .   . 
 .   . 
.L. ...
... ...

... ...
... ...
.DD .DD
... ...
 .   . 
 .   . 
.L. ...
..D ..D

... ...
... ...
.DD .DD
... ...
 .   . 
 .   . 
.LD ..D
... ...

... ...
... ...
.DD .DD
... ...
 .   . 
 .   . 
.LD ..D
..D ..D

... ...
... ...
.DD .DD
..D ..D
 .   . 
 .   . 
.L. ...
... ...

... ...
... ...
.DD .DD
..D ..D
 .   . 
 .   . 
.L. ...
..D ..D

... ...
... ...
.DD .DD
..D ..D
 .   . 
 .   . 
.LD ..D
... ...

... ...
... ...
.DD .DD
.

In [18]:
pd.Series(lut_values_current).mean(), pd.Series(lut_values_next).mean()

(0.9710044412117844, 1.0)

In [51]:
# dump the lut_transition table to parquet
do_query("COPY lut_transition TO 'lut_transition.parquet' (FORMAT PARQUET);")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,1936996626


In [54]:
# dump the lut_transition table to parquet
do_query("COPY boardstate_lut_expanded_int_values TO 'boardstates.parquet' (FORMAT PARQUET);")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count
0,137892016


In [47]:
do_query("SELECT * FROM boardstate_lut_expanded_int_values WHERE light_score = 7 LIMIT 10")

,lut_value,lut_index,light_score,dark_score,boardstate,A1,B1,C1,A2,B2,...,A7,B7,C7,A8,B8,C8,light_piece_count,dark_piece_count,light_piece_left_to_play,dark_piece_left_to_play
0,1.0,33555008,7,5,................D.......,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
1,1.0,33555040,7,4,................D......D,0,0,0,0,0,...,0,0,0,0,0,2,0,2,0,1
2,1.0,33555024,7,4,................D...D...,0,0,0,0,0,...,0,0,2,0,0,0,0,2,0,1
3,1.0,33555016,7,4,...........D....D.......,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,1
4,1.0,33555048,7,3,...........D....D......D,0,0,0,0,0,...,0,0,0,0,0,2,0,3,0,1
5,1.0,33555032,7,3,...........D....D...D...,0,0,0,0,0,...,0,0,2,0,0,0,0,3,0,1
6,1.0,33555064,7,2,...........D....D...D..D,0,0,0,0,0,...,0,0,2,0,0,2,0,4,0,1
7,1.0,33555012,7,4,........D.......D.......,0,0,0,0,0,...,0,0,0,0,0,0,0,2,0,1
8,1.0,33555044,7,3,........D.......D......D,0,0,0,0,0,...,0,0,0,0,0,2,0,3,0,1
9,1.0,33555060,7,2,........D.......D...D..D,0,0,0,0,0,...,0,0,2,0,0,2,0,4,0,1


In [49]:
do_query("SELECT * FROM boardstate_lut_expanded_int_values WHERE lut_index = 41943616")

,lut_value,lut_index,light_score,dark_score,boardstate,A1,B1,C1,A2,B2,...,A7,B7,C7,A8,B8,C8,light_piece_count,dark_piece_count,light_piece_left_to_play,dark_piece_left_to_play
0,0.931472,41943616,6,5,................D.L.....,0,0,0,0,0,...,1,0,0,0,0,0,1,1,0,1


In [11]:
# show boardstate for lut_index = 8575264
#do_query("SELECT * FROM boardstate_lut_expanded_int_values WHERE lut_index = 8575264")

"""
.D.
.D.
...
...
...
...
L..
..D
"""

"""
.D.
.D.
...
...
...
...
...
..D
"""
do_query("SELECT * FROM boardstate_lut_expanded_int_values WHERE lut_index = 186656")


,lut_value,lut_index,light_score,dark_score,boardstate,A1,B1,C1,A2,B2,...,A7,B7,C7,A8,B8,C8,light_piece_count,dark_piece_count,light_piece_left_to_play,dark_piece_left_to_play
0,1.0,186656,7,4,.D..D..................D,0,2,0,0,2,...,0,0,0,0,0,2,0,3,0,0


In [5]:
do_query("CREATE INDEX lut_transition_current_state ON lut_transition(current_state);")
do_query("CREATE INDEX lut_transition_next_state ON lut_transition(next_state);")


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count


In [49]:
do_query("CREATE UNIQUE INDEX lut_index_index ON boardstate_lut_expanded_int_values(lut_index)")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,Count


In [34]:
print(do_query("""
EXPLAIN SELECT * FROM lut_transition WHERE current_state = 8563407 LIMIT 10
""").explain_value[0])

┌───────────────────────────┐
│      STREAMING_LIMIT      │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│           FILTER          │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│ (current_state = 8563407) │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│        EC: 75489052       │
└─────────────┬─────────────┘                             
┌─────────────┴─────────────┐
│        INDEX_SCAN         │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│       lut_transition      │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│       current_state       │
│         next_state        │
│            dice           │
│        next_player        │
│   ─ ─ ─ ─ ─ ─ ─ ─ ─ ─ ─   │
│       EC: 377445263       │
└───────────────────────────┘                             



In [45]:
u = do_query("""
    SELECT * FROM lut_transition
    JOIN (SELECT lut_index FROM boardstate_lut_expanded_int_values LIMIT 10) as t
    ON lut_transition.current_state = t.lut_index
""")

print(u)

    current_state  next_state  dice  next_player  lut_index
0         8563407     8563407     0            2    8563407
1         8563407      174799     1            2    8563407
2         8563407     8542671     1            1    8563407
3         8563407     8561103     1            2    8563407
4         8563407    42114575     2            2    8563407
..            ...         ...   ...          ...        ...
94        8563516    42114684     2            2    8563516
95        8563516     8563388     3            2    8563516
96        8563516     8533564     3            2    8563516
97        8563516    42087036     4            2    8563516
98        8563516    25337276     4            2    8563516

[99 rows x 5 columns]


In [47]:
# list index on boardstate_lut_expanded_int_values table
do_query('SELECT sql FROM duckdb_indexes()')

,sql


In [14]:
# check type of lut_index column
type(do_query("""
SELECT lut_index FROM boardstate_lut_expanded_int_values LIMIT 1
""").lut_index[0])

numpy.int32

In [15]:
type(do_query("""
SELECT current_state FROM lut_transition LIMIT 1
""").current_state[0])

numpy.int32